In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [7]:
train_dir = 'C:/Users/RAJVI/MACHINE LEARNING/multiple_LR/cancer/images/train'  
validation_dir = 'C:/Users/RAJVI/MACHINE LEARNING/multiple_LR/cancer/images/test'  


In [8]:
#  image preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [9]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' if you have more than 2 classes
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' if you have more than 2 classes
)

Found 6739 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


In [10]:
# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [11]:
# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(9, activation='softmax')(x)  # Use Dense(number_of_classes, activation='softmax') for multi-class


In [13]:
# Create the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for multi-class
              metrics=['accuracy'])

In [14]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=3,  # Increase this for better training
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


c:\Users\RAJVI\anaconda3\envs\pin\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1337 - loss: 2.3997

c:\Users\RAJVI\anaconda3\envs\pin\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 652s 3s/step - accuracy: 0.1337 - loss: 2.3989 - val_accuracy: 0.1354 - val_loss: 2.2227
Epoch 2/3
  1/210 ━━━━━━━━━━━━━━━━━━━━ 10:41 3s/step - accuracy: 0.1562 - loss: 2.1834

c:\Users\RAJVI\anaconda3\envs\pin\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


210/210 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.1562 - loss: 2.1834 - val_accuracy: 0.1364 - val_loss: 2.2390
Epoch 3/3
210/210 ━━━━━━━━━━━━━━━━━━━━ 660s 3s/step - accuracy: 0.1697 - loss: 2.1494 - val_accuracy: 0.1562 - val_loss: 2.1512


In [15]:
# Save the model
model.save('melanoma_transfer_learning_model.h5')




In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.1683 - loss: 2.1396
Validation Accuracy: 0.1525423675775528
